In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px

In [ ]:
from vega_datasets import data


In [88]:
%%writefile app.py
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px

# Page Configuration
st.set_page_config(page_title="US Population Dashboard", page_icon="🏂", layout="wide")

alt.themes.enable("dark")

# Custom CSS
st.markdown("""
<style>
[data-testid="block-container"] { padding: 2rem; }
[data-testid="stMetric"] { background-color: #393939; text-align: center; padding: 15px; }
</style>
""", unsafe_allow_html=True)

# Sample Data
st.write("Streamlit app content goes here...")


Writing app.py


In [7]:
!streamlit run app.py


^C


In [89]:
merged_df = pd.read_csv('C:\Tweeter project\Election dataset\merged_df.csv')


In [94]:
merged_df.shape

(1299, 10)

In [ ]:
merged_df['state'].unique()

In [ ]:
import altair as alt

alt.themes.enable("dark")
# Convert the 'date' column to a string format
merged_df['date'] = merged_df['date'].astype(str)

# Create the heatmap chart
heatmap = alt.Chart(merged_df).mark_rect().encode(
    y=alt.Y('date:O', axis=alt.Axis(title="Date", titleFontSize=16, titlePadding=15, titleFontWeight=900, labelAngle=0)),
    x=alt.X('state:O', axis=alt.Axis(title="State", titleFontSize=16, titlePadding=15, titleFontWeight=900)),
    color=alt.Color('max(total_likes):Q',
                    legend=alt.Legend(title="Total Likes"),
                    scale=alt.Scale(scheme="blueorange")),
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.25),
).properties(width=1000).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)

heatmap

In [ ]:
import altair as alt
from vega_datasets import data


alt.themes.enable("dark")

# Load U.S. states boundaries (topojson)
state = alt.topo_feature(data.us_10m.url, 'states')

# Create the chart
chart = alt.Chart(state).mark_geoshape().encode(
    color=alt.Color('total_likes:Q', scale=alt.Scale(scheme='blues')),   # color based on total_likes
    stroke=alt.value('#154360')
).transform_lookup(
    lookup='id',  # matching on 'id' in the topojson data
    from_=alt.LookupData(merged_df, 'id', ['total_likes'])  # Lookup total_likes from merged_df by 'id'
).properties(
    width=700,
    height=400
).project(
    type='albersUsa'  # Use Albers USA projection
)

chart


# alt.themes.enable("dark")

# # Load U.S. states boundaries (topojson)
# # Step 1: Aggregate data to get the total likes per state
# state_likes = merged_df.groupby('id', as_index=False).agg({'total_likes': 'sum'})

# # Step 2: Create the Altair chart using the aggregated data
# chart = alt.Chart(state).mark_geoshape().encode(
#     color=alt.Color('total_likes:Q', scale=alt.Scale(scheme='blues')),  # color based on total_likes
#     stroke=alt.value('#154360')
# ).transform_lookup(
#     lookup='id',  # matching on 'id' in the topojson data
#     from_=alt.LookupData(state_likes, 'id', ['total_likes'])  # Lookup total_likes from state_likes by 'id'
# ).properties(
#     width=700,
#     height=400
# ).project(
#     type='albersUsa'  # Use Albers USA projection
# )

# chart

alt.Chart(...)

In [30]:
import plotly.express as px



# # Create the choropleth map
# choropleth = px.choropleth(
#     merged_df,
#     locations='states_code',  # Ensure this contains full state names, or map abbreviations to names
#     color='tweet_count',
#     locationmode="USA-states",
#     color_continuous_scale='blues',
#     range_color=(0, max(merged_df.tweet_count)),  # Set the range for color scaling
#     scope="usa",  # Focus on the United States
#     labels={'tweet_count': 'tweet count'}
#                                 )
# choropleth.update_layout(
#     template='plotly_dark',  # Dark theme
#     plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
#     paper_bgcolor='rgba(0, 0, 0, 0)',  # Transparent paper background
#     margin=dict(l=0, r=0, t=0, b=0),  # Remove margins for a tight fit
#     height=450  # Adjust the height of the map
# )

# # Show the map
# choropleth

# Step 1: Aggregate data to get the total tweet count per state
state_totals = merged_df.groupby('states_code', as_index=False).agg({'tweet_count': 'sum'})

# Step 2: Create the choropleth map using the aggregated data
choropleth = px.choropleth(
    state_totals,
    locations='states_code',  # Use state codes (two-letter abbreviations)
    color='tweet_count',
    locationmode="USA-states",
    color_continuous_scale='blues',
    range_color=(0, max(state_totals.tweet_count)),  # Adjust color scaling based on aggregated data
    scope="usa",
    labels={'tweet_count': 'Total Tweet Count'}  # Label for the color bar
)

# Step 3: Update layout for styling
choropleth.update_layout(
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=0, b=0),
    width=900,
    height=500
)

# Show the map
choropleth

In [35]:
import plotly.graph_objects as go

# Step 1: Aggregate data to get the total tweet count and total likes per state
state_totals = merged_df.groupby('states_code', as_index=False).agg({
    'tweet_count': 'sum',
    'total_likes': 'sum'
})

# Step 2: Initialize the base figure using go.Figure
fig = go.Figure()

# Step 3: Add a choropleth trace for Total Tweet Count
fig.add_trace(go.Choropleth(
    locations=state_totals['states_code'],
    z=state_totals['tweet_count'],
    locationmode="USA-states",
    colorscale="Blues",
    colorbar_title="Total Tweet Count",
    visible=True,  # Set this trace to be visible by default
    hoverinfo="location+z",  # Only show location and value (tweet count)
    hovertemplate="<b>%{location}</b><br>Total Tweet Count: %{z}<extra></extra>"  # Custom hover text
))

# Step 4: Add a choropleth trace for Total Likes
fig.add_trace(go.Choropleth(
    locations=state_totals['states_code'],
    z=state_totals['total_likes'],
    locationmode="USA-states",
    colorscale="Greens",  # Different color scale for total likes
    colorbar_title="Total Likes",
    visible=False,  # This trace will be hidden by default
    hoverinfo="location+z",  # Only show location and value (total likes)
    hovertemplate="<b>%{location}</b><br>Total Likes: %{z}<extra></extra>"  # Custom hover text
))

# Step 5: Add the dropdown menu for selecting between Total Tweet Count and Total Likes
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": "Total Tweet Count",
                    "method": "update",
                    "args": [
                        {"visible": [True, False]},  # Show tweet count trace, hide likes trace
                        {"coloraxis": {"colorbar": {"title": "Total Tweet Count"}}}
                    ]
                },
                {
                    "label": "Total Likes",
                    "method": "update",
                    "args": [
                        {"visible": [False, True]},  # Show likes trace, hide tweet count trace
                        {"coloraxis": {"colorbar": {"title": "Total Likes"}}}
                    ]
                }
            ],
            "direction": "down",
            "showactive": True,
            "x": 0.1,
            "xanchor": "left",
            "y": 1.15,
            "yanchor": "top"
        }
    ],
    geo_scope='usa',  # Limit map scope to the USA
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=0, b=0),
    height=450
)

# Show the figure
fig.show()


In [36]:
import plotly.graph_objects as go
import pandas as pd

# Step 1: Aggregate data by state, tweet count, and total likes, and get unique dates
state_totals = merged_df.groupby(['date', 'states_code'], as_index=False).agg({
    'tweet_count': 'sum',
    'total_likes': 'sum'
})
available_dates = state_totals['date'].unique()

# Step 2: Set initial date (e.g., the first date in the dataset)
initial_date = available_dates[0]
filtered_data = state_totals[state_totals['date'] == initial_date]

# Step 3: Initialize the base figure with initial date's data
fig = go.Figure()

# Add choropleth trace for Total Tweet Count
fig.add_trace(go.Choropleth(
    locations=filtered_data['states_code'],
    z=filtered_data['tweet_count'],
    locationmode="USA-states",
    colorscale="Blues",
    colorbar_title="Total Tweet Count",
    visible=True,
    hoverinfo="location+z",
    hovertemplate="<b>%{location}</b><br>Total Tweet Count: %{z}<extra></extra>"
))

# Add choropleth trace for Total Likes
fig.add_trace(go.Choropleth(
    locations=filtered_data['states_code'],
    z=filtered_data['total_likes'],
    locationmode="USA-states",
    colorscale="Greens",
    colorbar_title="Total Likes",
    visible=False,
    hoverinfo="location+z",
    hovertemplate="<b>%{location}</b><br>Total Likes: %{z}<extra></extra>"
))

# Step 4: Update layout with dropdown menus for metrics and date filtering
fig.update_layout(
    updatemenus=[
        # Metric selection menu
        {
            "buttons": [
                {
                    "label": "Total Tweet Count",
                    "method": "update",
                    "args": [
                        {"visible": [True, False]},
                        {"coloraxis": {"colorbar": {"title": "Total Tweet Count"}}}
                    ]
                },
                {
                    "label": "Total Likes",
                    "method": "update",
                    "args": [
                        {"visible": [False, True]},
                        {"coloraxis": {"colorbar": {"title": "Total Likes"}}}
                    ]
                }
            ],
            "direction": "down",
            "showactive": True,
            "x": 0.1,
            "xanchor": "left",
            "y": 1.15,
            "yanchor": "top"
        },
        # Date filter menu
        {
            "buttons": [
                {
                    "label": str(date),
                    "method": "update",
                    "args": [
                        {
                            "z": [state_totals[state_totals['date'] == date]['tweet_count'].values,
                                  state_totals[state_totals['date'] == date]['total_likes'].values]
                        }
                    ]
                } for date in available_dates
            ],
            "direction": "down",
            "showactive": True,
            "x": 0.02,
            "xanchor": "left",
            "y": 1.15,
            "yanchor": "top"
        }
    ],
    geo_scope='usa',
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=0, b=0),
    height=450
)

# Show the figure
fig.show()


In [78]:
import plotly.graph_objects as go
import pandas as pd

# Step 1: Aggregate data by state, tweet count, and total likes, and get unique dates
state_totals = merged_df.groupby(['date', 'states_code'], as_index=False).agg({
    'tweet_count': 'sum',
    'total_likes': 'sum'
})
available_dates = state_totals['date'].unique()

# Step 2: Set initial date (e.g., the first date in the dataset)
initial_date = available_dates[0]
filtered_data = state_totals[state_totals['date'] == initial_date]

# Step 3: Initialize the base figure with initial date's data
fig = go.Figure()

# Add choropleth trace for Total Tweet Count
fig.add_trace(go.Choropleth(
    locations=filtered_data['states_code'],
    z=filtered_data['tweet_count'],
    locationmode="USA-states",
    colorscale="Blues",
    colorbar_title="Total Tweet Count",
    visible=True,
    hoverinfo="location+z",
    hovertemplate="<b>%{location}</b><br>Total Tweet Count: %{z}<extra></extra>"
))

# Add choropleth trace for Total Likes
fig.add_trace(go.Choropleth(
    locations=filtered_data['states_code'],
    z=filtered_data['total_likes'],
    locationmode="USA-states",
    colorscale="Greens",
    colorbar_title="Total Likes",
    visible=False,
    hoverinfo="location+z",
    hovertemplate="<b>%{location}</b><br>Total Likes: %{z}<extra></extra>"
))

# Step 4: Update layout with dropdown menus for metrics and date filtering
fig.update_layout(
    updatemenus=[
        # Metric selection menu
        {
            "buttons": [
                {
                    "label": "Total Tweet Count",
                    "method": "update",
                    "args": [
                        {"visible": [True, False]},
                        {"coloraxis": {"colorbar": {"title": "Total Tweet Count"}}}
                    ]
                },
                {
                    "label": "Total Likes",
                    "method": "update",
                    "args": [
                        {"visible": [False, True]},
                        {"coloraxis": {"colorbar": {"title": "Total Likes"}}}
                    ]
                }
            ],
            "direction": "down",
            "showactive": True,
            "x": 1,
            "xanchor": "left",
            "y": 1.20,
            "yanchor": "top"
        },
        # Date filter menu
        {
            "buttons": [
                {
                    "label": str(date),
                    "method": "update",
                    "args": [
                        {
                            "visible": [True, False],  # Ensure visibility of the two traces
                            "z": [
                                state_totals[state_totals['date'] == date]['tweet_count'].values,
                                state_totals[state_totals['date'] == date]['total_likes'].values
                            ]
                        },
                        {"coloraxis": {"colorbar": {"title": "Total Tweet Count"}}}
                    ]
                } for date in available_dates
            ],
            "direction": "down",
            "showactive": True,
            "x": 0.01,
            "xanchor": "center",
            "y": 1.33,
            "yanchor": "top"
        }
    ],
    geo_scope='usa',
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=0, b=0),
    height=450
)

# Show the figure
fig.show()


In [ ]:
pip install streamlit

In [85]:
import streamlit as st
import plotly.graph_objects as go
import pandas as pd



# Step 1: Aggregate data by state, tweet count, and total likes, and get unique dates
state_totals = merged_df.groupby(['date', 'states_code'], as_index=False).agg({
    'tweet_count': 'sum',
    'total_likes': 'sum'
})
available_dates = state_totals['date'].unique()

# Step 2: Set initial date (e.g., the first date in the dataset)
initial_date = available_dates[0]
filtered_data = state_totals[state_totals['date'] == initial_date]

# Step 3: Initialize the base figure with initial date's data
fig = go.Figure()

# Add choropleth trace for Total Tweet Count
fig.add_trace(go.Choropleth(
    locations=filtered_data['states_code'],
    z=filtered_data['tweet_count'],
    locationmode="USA-states",
    colorscale="Blues",
    colorbar_title="Total Tweet Count",
    visible=True,
    hoverinfo="location+z",
    hovertemplate="<b>%{location}</b><br>Total Tweet Count: %{z}<extra></extra>"
))

# Add choropleth trace for Total Likes
fig.add_trace(go.Choropleth(
    locations=filtered_data['states_code'],
    z=filtered_data['total_likes'],
    locationmode="USA-states",
    colorscale="Greens",
    colorbar_title="Total Likes",
    visible=False,
    hoverinfo="location+z",
    hovertemplate="<b>%{location}</b><br>Total Likes: %{z}<extra></extra>"
))

# Step 4: Streamlit dropdown menus for selecting metrics and dates

# Metric selection dropdown
metric = st.selectbox(
    "Select Metric",
    ("Total Tweet Count", "Total Likes"),
    index=0  # Default to 'Total Tweet Count'
)

# Date selection dropdown
date_selected = st.selectbox(
    "Select Date",
    available_dates,
    index=0  # Default to the first date
)

# Filter data based on selected date
filtered_data_date = state_totals[state_totals['date'] == date_selected]

# Update the figure with selected metric and date
if metric == "Total Tweet Count":
    fig.data[0].visible = True  # Show tweet count trace
    fig.data[1].visible = False  # Hide likes trace
    fig.data[0].z = filtered_data_date['tweet_count']
    fig.data[0].colorbar.title = "Total Tweet Count"
elif metric == "Total Likes":
    fig.data[0].visible = False  # Hide tweet count trace
    fig.data[1].visible = True  # Show likes trace
    fig.data[1].z = filtered_data_date['total_likes']
    fig.data[1].colorbar.title = "Total Likes"

# Display the map with updated data
st.plotly_chart(fig, use_container_width=True)


2024-11-14 12:30:20.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 12:30:20.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 12:30:20.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 12:30:20.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 12:30:20.497 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 12:30:20.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 12:30:20.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 12:30:20.506 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [97]:
merged_df.head(3)

,state,date,tweet_count,total_likes,sentiment_avg,total_retweets,top_hashtag,top_hashtag_likes,id,states_code
0,Alabama,2020-10-15,44,277.0,-0.001450,92.0,#biden,226.0,1,AL
1,Alabama,2020-10-16,59,618.0,-0.080724,127.0,#trump,461.0,1,AL
2,Alabama,2020-10-17,29,142.0,0.143159,46.0,#trump,128.0,1,AL


In [99]:
merged_df['top_hashtag'].unique()

array(['#biden', '#trump', '#election2020', '#vote', '#awesomestuff',
       '#gop', '#2020election', '#resist', '#electionday',
       '#60minutesinterview', '#acb', '#trumpsawhinybitch', '#wtpblue',
       '#horrible', '#covid19', '#trumpvirus', '#democrat',
       '#madametussauds', '#california', '#hunterbiden'], dtype=object)